In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import time
import os.path

import shapely.geometry as geom
import geopandas as gpd
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.geometry import Point

In [4]:
link_base = 'link.csv'
# TMC_file = 'TMC_Identification_I10.csv'
# TMC_file = 'TMC_Identification_loop_303.csv'
TMC_file = 'TMC_Identification_US60.csv'
'''reading link_base'''
link_base = pd.read_csv(link_base, low_memory=False)
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
# link_base = link_base[-link_base['name'].isna()]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
# link_base = link_base.reset_index()
# link_base = link_base.drop(['index'], 1)

'''convert link_base to MultiLineString''' 
multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

from shapely.geometry import MultiLineString
line_base = MultiLineString(multiline_string_base_list) 

'''reading tmc'''
tmc = pd.read_csv(TMC_file)
tmc = tmc.drop_duplicates(subset=['direction','road_order']).sort_values(by=['direction','road_order'])
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
origin_tmc_num = len(tmc)

'''remove out boundary tmc'''
in_bbox_index_list = []
for i in tmc.index:
    if (tmc['start_longitude'][i] > line_base.bounds[0]) & (tmc['start_longitude'][i] < line_base.bounds[2]) & \
        (tmc['end_longitude'][i] > line_base.bounds[0]) & (tmc['end_longitude'][i] < line_base.bounds[2]) & \
            (tmc['start_latitude'][i] > line_base.bounds[1]) & (tmc['start_latitude'][i] < line_base.bounds[3]) & \
        (tmc['end_latitude'][i] > line_base.bounds[1]) & (tmc['end_latitude'][i] < line_base.bounds[3]):
        in_bbox_index_list.append(i)

tmc = tmc.loc[in_bbox_index_list]
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
if len(in_bbox_index_list) < origin_tmc_num:
    print('base map cannot cover all TMC nodes,' + str(origin_tmc_num-len(in_bbox_index_list)) + 'tmc nodes are out of boundary box, please use larger base map')

form_tmc = pd.DataFrame([])
gp = tmc.groupby(['road','direction'])
for key, form in gp:
    form['end_latitude'][:-1] = form['start_latitude'][1:].tolist()
    form['end_longitude'][:-1] = form['start_longitude'][1:].tolist()
    form_tmc = pd.concat([form_tmc, form], axis=0)
form_tmc = form_tmc.reset_index()
form_tmc = form_tmc.drop(['index'], 1)
tmc = form_tmc


'''build node.csv'''
print('converting tmc data into gmns format...')
p=1

node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['geometry'] = None

gp = tmc.groupby(['road','direction'])
for key, form in gp:
    for i in form.index:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)



# for i in range(0,len(tmc)-1):
#     if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
#         node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
#                                     'x_coord': tmc.loc[i,'start_longitude'], \
#                                     'y_coord': tmc.loc[i,'start_latitude'],\
#                                     'z_coord': None,\
#                                     'node_type': 'tmc_start',\
#                                     'ctrl_type': None,\
#                                     'zone_id': None,\
#                                     'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
#     else:
#         node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
#                                     'x_coord': tmc.loc[i,'start_longitude'], \
#                                     'y_coord': tmc.loc[i,'start_latitude'],\
#                                     'z_coord': None,\
#                                     'node_type': 'tmc_start',\
#                                     'ctrl_type': None,\
#                                     'zone_id': None,\
#                                     'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
#         node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
#                                     'x_coord': tmc.loc[i,'end_longitude'], \
#                                     'y_coord': tmc.loc[i,'end_latitude'],\
#                                     'z_coord': None,\
#                                     'node_type': 'tmc_end',\
#                                     'ctrl_type': None,\
#                                     'zone_id': None,\
#                                     'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)

#     if i > p/10 * len(tmc): 
#         print(str(p*10)+"%"+' nodes completed!')
#         p = p + 1

# node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
#                                     'x_coord': tmc.loc[i+1,'start_longitude'], \
#                                     'y_coord': tmc.loc[i+1,'start_latitude'],\
#                                     'z_coord': None,\
#                                     'node_type': 'tmc_start',\
#                                     'ctrl_type': None,\
#                                     'zone_id': None,\
#                                     'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

# node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    # 'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    # 'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    # 'z_coord': None,\
                                    # 'node_type': 'tmc_end',\
                                    # 'ctrl_type': None,\
                                    # 'zone_id': None,\
                                    # 'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv (' + str(len(node_tmc)) + ' nodes' + ') generated!')


'''build link_tmc.csv'''
p = 1
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                # 'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                # 'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude'])&(node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'from_node_id': node_tmc[(node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' links completed!')
        p = p + 1
        
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001

link_tmc.to_csv('link_tmc.csv')

print('link_tmc.csv (' + str(len(link_tmc)) + ' links' + ') generated!')

converting tmc data into gmns format...
node_tmc.csv (32 nodes) generated!
10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (28 links) generated!


<ipython-input-4-c6275bf2b9f4>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form['end_latitude'][:-1] = form['start_latitude'][1:].tolist()
/usr/local/home/ysx28/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-4-c6275bf2b9f4>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form['end_longitude'][:-1] = form['

In [4]:
import network_matching_function as nmf 
# nmf.TMCIdentification2GMNSNodeLinkFiles('TMC_Identification_I10.csv','link.csv')
# nmf.TMCIdentification2GMNSNodeLinkFiles('TMC_Identification_loop_303.csv','link.csv')
nmf.TMCIdentification2GMNSNodeLinkFiles('TMC_Identification_US60.csv','link.csv')

/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/AZNetwork/network_matching_function.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form['end_latitude'][:-1] = form['start_latitude'][1:].tolist()
/usr/local/home/ysx28/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/AZNetwork/network_matching_function.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

converting tmc data into gmns format...
node_tmc.csv (67 nodes) generated!
10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (63 links) generated!


In [4]:
nmf.ConvertTMCReading2Measurement('Reading.csv','link_tmc.csv')


10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!


In [5]:
nmf.MatchTMC2GMNSNetwork('link_tmc.csv','link.csv')

10% initial matching completed!
20% initial matching completed!
30% initial matching completed!


In [6]:
nmf.ConvertMeasurementBasedOnMatching('link.csv','matching_tmc2gmns.csv','measurement_tmc.csv')

10% measurement_base completed!
20% measurement_base completed!
30% measurement_base completed!
40% measurement_base completed!
50% measurement_base completed!
60% measurement_base completed!
70% measurement_base completed!
80% measurement_base completed!
90% measurement_base completed!
100% measurement_base completed!
measurement_base.csv generated!


In [5]:
#intergrate all functions in one

import network_matching_function as nmf 
nmf.OneFunction()

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (149 nodes) generated!
10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (144 links) generated!
10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!
10% initial matching completed!
20% initial matching completed!
30% initial matching completed!
40% initial matching completed!
50% initial matching completed!
60% initial matching completed!
70% init